# Parameter Fitting for Empirical Flow Stress Models — Sellars-Tegart Model

The theory behind is here discussed only very briefly, see Solhjoo2022 <https://doi.org/10.3390/modelling3030023> for detailed information.

In [ ]:
# only for Google Colab
!wget -N "https://raw.githubusercontent.com/Institute-of-Metal-Forming/notebooks-ilsenburg/refs/heads/main/requirements.txt"
!wget -N "https://raw.githubusercontent.com/Institute-of-Metal-Forming/notebooks-ilsenburg/refs/heads/main/flow-stress/bst.csv"
!pip install -r "requirements.txt"

Import of required packages.

In [ ]:
import pandas as pd  # for data loading
import numpy as np  # for vectorized computations
from numpy.polynomial import Polynomial
import scipy.optimize as opt  # for least squares fitting
import scipy.interpolate as interp
import plotly.express as px  # for plotting

## Loading the Experimental Data

We have the flow stress data in a long format CSV file and load it via `pandas` into a data frame.

In [ ]:
data = pd.read_csv(
    "bst.csv",  # file name
    encoding="utf8",  # use unicode to be safe on Windows systems (default on UNIX)
    sep=",",  # columns separated by comma
)
data

As the data is in long format (columns for temperature, strain and strain rate; rows for every data point), it is very comfortable to plot it with `plotly`.
But first we sort by temperature, then by strain rate and last by strain.

In [ ]:
data.sort_values(
    by=["temp", "rate", "strain"],  # multi-level sorting columns
    ascending=True,  # order from small to large
    inplace=True,  # use the existing data frame, do not copy the data
)
data

Now we can plot the data in dependence of the test conditions.

In [ ]:
px.line(
    data,
    "strain",  # strain on the x-axis
    "stress",  # stress on the y-axis
    color="temp",  # distinguish temperatures by colors
    facet_col="rate",  # draw multiple plots for distinct strain rates
    line_group="file",  # distinguish lines by file name, avoid zick-zack curves
)

In [ ]:
data.groupby([""])

## The Sellars-Tegart Flow Stress Model

For low stresses, the model is given as below, with $A'$ and $n'$ as parameters.

$$ Z = A' \sigma^{n'} $$

For high stresses, the model is given as below, with $A''$ and $\beta$ as parameters.

$$ Z = A'' \exp(\beta\sigma) $$

A merged approach is given as below, with $A$, $\alpha$ and $n$ as parameters.

$$ Z = A [ \sinh ( \alpha\sigma )]^n $$

The Zener-Holomon parameter occuring in all approaches is defined as below, with an activation energy $Q$.

$$ Z = \dot{\varphi} \exp \left( \frac{Q}{RT} \right)$$

## Classic 6-Step Determination of Parameters

### Interpolation of Flow Stress Data

The following procedures need data at defined strains, so we first interpolate the given data.

In [ ]:
interpolations = data.groupby(["temp", "rate", "file"]).apply(
    lambda d: interp.make_interp_spline(
        d["strain"],
        d["stress"],
        k=1,
    ),
    include_groups=False,
)

Then, we evaluate the interpolations at defined points.

In [ ]:
strains = np.linspace(0.1, 1.2, 100)

In [ ]:
interpolated_data = (
    interpolations.apply(lambda ip: pd.Series(ip(strains), index=strains))
    .reset_index()
    .melt(id_vars=["temp", "rate", "file"], var_name="strain", value_name="stress")
)
interpolated_data

The plots should resemble the previous ones (good if solid and dashed lines are hardly distinguishable).

In [ ]:
px.line(
    pd.concat(
        [data, interpolated_data],
        keys=["orig", "interp"],
        names=["source", "index"],
    ).reset_index(level=0),
    "strain",  # strain on the x-axis
    "stress",  # stress on the y-axis
    color="temp",  # distinguish temperatures by colors
    facet_col="rate",  # draw multiple plots for distinct strain rates
    line_dash="source",
    line_group="file",
)

### Calculation of $n'$

$n'$ is obtained from the slope of $\ln(\dot\varphi)$ over $\ln(\sigma)$ for each strain and temperature. Usually, the mean value of all temperatures is taken.

In [ ]:
nprime = interpolated_data.groupby("strain").apply(
    lambda sg: sg.groupby("temp").apply(
        lambda tg: opt.least_squares(
            lambda params: Polynomial(params)(np.log(tg["stress"]))
            - np.log(tg["rate"]),
            x0=[0, 1],
        ).x[1],
        include_groups=False,
    ),
    include_groups=False,
)
nprime["mean"] = nprime.agg("mean", axis=1)
nprime

Plot the values of $n'$ in dependence on strain and temperature.

In [ ]:
px.line(nprime, labels=dict(value="n'"))

### Calculation of $\beta$

$\beta$ is obtained from the slope of $\ln(\dot\varphi)$ over $\sigma$ for each strain and temperature. Usually, the mean value of all temperatures is taken.

In [ ]:
beta = interpolated_data.groupby("strain").apply(
    lambda sg: sg.groupby("temp").apply(
        lambda tg: opt.least_squares(
            lambda params: Polynomial(params)(tg["stress"]) - np.log(tg["rate"]),
            x0=[0, 1],
        ).x[1],
        include_groups=False,
    ),
    include_groups=False,
)
beta["mean"] = beta.agg("mean", axis=1)
beta

Plot the values of $\beta$ in dependence on strain and temperature.

In [ ]:
px.line(beta, labels=dict(value="beta"))

### Calculation of $\alpha$

$\alpha$ is obtained by dividing $\beta$ and $n'$. Usually, the mean value of all temperatures is taken.

In [ ]:
alpha = beta / nprime
alpha

Plot the values of $\alpha$ in dependence on strain.

In [ ]:
px.line(alpha, labels=dict(value="alpha"))

The overall mean of $\alpha$ is used as a constant parameter.

In [ ]:
alpha_mean = np.mean(alpha["mean"])
alpha_mean

### Calculation of $n$

With the mean $\alpha$ given, $N$ is obtained from the slope of $\ln(\dot\varphi)$ over $\ln(\sinh(\alpha\sigma))$ for each strain and temperature. Usually, the mean value of all temperatures is taken.

In [ ]:
n = interpolated_data.groupby("strain").apply(
    lambda sg: sg.groupby("temp").apply(
        lambda tg: opt.least_squares(
            lambda params: Polynomial(params)(
                np.log(np.sinh(alpha_mean * tg["stress"]))
            )
            - np.log(tg["rate"]),
            x0=[0, 1],
        ).x[1],
        include_groups=False,
    ),
    include_groups=False,
)
n["mean"] = n.agg("mean", axis=1)
n

Plot the values of $n'$ in dependence on strain and temperature.

In [ ]:
px.line(n, labels=dict(value="beta"))

The overall mean of $n$ is used as a constant parameter.

In [ ]:
n_mean = np.mean(n["mean"])
n_mean

## Define the Model Functions

The activation energy $Q$ is usually modelled as a polynomial in dependence on the strain (here of 9th order).

In [ ]:
GAS_CONSTANT = 8.314

In [ ]:
def activation_energy_poly(params):
    return np.polynomial.Polynomial(params[0:10], symbol="φ")

Similarly the pre-factor $A$.

In [ ]:
def pre_factor_poly(params):
    return np.polynomial.Polynomial(params[10:20], symbol="φ")

We have two possibilities to fit the model. We can use the parameters $\alpha$ and $n$ as determined above (6-step method) or we can leave them free and fit them alongside using least squares (full method).

In the 6-step case we have only the free parameters of the two polynomials (20 in total) collected in the `params` array.

In [ ]:
def sellars_tegart_model_6step(strain, rate, temp, params):
    zener = rate * np.exp(
        activation_energy_poly(params)(strain) / (GAS_CONSTANT * temp)
    )
    return (
        1
        / alpha_mean
        * np.sinh(zener / pre_factor_poly(params)(strain)) ** (1 / n_mean)
    )

In the full case we have two more parameters (22 in total) collected in the `params` array.

In [ ]:
def sellars_tegart_model_full(strain, rate, temp, params):
    zener = rate * np.exp(
        activation_energy_poly(params)(strain) / (GAS_CONSTANT * temp)
    )
    return (
        1
        / params[20]
        * np.sinh(zener / pre_factor_poly(params)(strain)) ** (1 / params[21])
    )

## Fit the Model to the Data

Now we use the leat squares method to find an optimal fit of the model to the data (implemented in `scipy.optimize.least_squares`).
We define a function that computes the absolute error between the data and the model at every data point for a given set of model parameters (`params`).
Then, we pass it to the optimization routine which returns the optimal fit to us.

### 6-step Method

In [ ]:
fit_6step = opt.least_squares(
    lambda params: sellars_tegart_model_6step(data.strain, data.rate, data.temp, params)
    - data.stress,
    x0=np.concat(
        [
            np.full(10, 1),
            np.full(10, 1),
        ]
    ),  # initial guess of the params vector
    max_nfev=10_000,
)
fit_6step

### Full Method (may run for long time)

In [ ]:
fit_full = opt.least_squares(
    lambda params: sellars_tegart_model_full(data.strain, data.rate, data.temp, params)
    - data.stress,
    x0=np.concat(
        [
            np.full(10, 1),
            np.full(10, 1),
            [5e-3, 5],
        ]
    ),  # initial guess of the params vector
    max_nfev=10_000,
)
fit_full

## Investigate the Fit Results

First, define a space of strains to evaluate the polynomials on.

In [ ]:
strains = np.linspace(0, 1.5, 200)

### Full Method

#### Activation Energy $Q$

The fit gives the following polynomial for the activation energy $Q$.

In [ ]:
print("Q = ", activation_energy_poly(fit_6step.x))

Plot it.

In [ ]:
px.line(
    x=strains,
    y=activation_energy_poly(fit_6step.x)(strains),
    labels=dict(x="strain", y="activation_energy"),
)

#### Pre-Factor $A$

The fit gives the following polynomial for the pre-factor $A$.

In [ ]:
print("A = ", pre_factor_poly(fit_6step.x))

Plot it.

In [ ]:
px.line(
    x=strains,
    y=pre_factor_poly(fit_6step.x)(strains),
    labels=dict(x="strain", y="pre_factor"),
)

### 6-step Method

### Full Method

#### Activation Energy $Q$

The fit gives the following polynomial for the activation energy $Q$.

In [ ]:
print("Q = ", activation_energy_poly(fit_full.x))

Plot it.

In [ ]:
px.line(
    x=strains,
    y=activation_energy_poly(fit_full.x)(strains),
    labels=dict(x="strain", y="activation_energy"),
)

#### Pre-Factor $A$

The fit gives the following polynomial for the pre-factor $A$.

In [ ]:
print("A = ", pre_factor_poly(fit_full.x))

Plot it.

In [ ]:
px.line(
    x=strains,
    y=pre_factor_poly(fit_full.x)(strains),
    labels=dict(x="strain", y="pre_factor"),
)

## Plot the Model Predictions Counter the Data

First we create a fine cartesian raster to evaluate the model on, so we get smooth curves of the model prediction.

In [ ]:
strains = np.linspace(0, 1.2, 50)  # strain with 50 points between 0 and 1.2
temps = (
    data.temp.unique()
)  # take only the distinct temperatures that are present in the data
rates = data.rate.unique()  # respectively

grid = pd.MultiIndex.from_product(
    [temps, rates, strains], names=["temp", "rate", "strain"]
)
grid

The raster is an index for a data frame, but we want it as an actual dataframe for easier computation.

In [ ]:
model_data_6step = pd.DataFrame(index=grid).reset_index()
model_data_6step

In [ ]:
model_data_full = pd.DataFrame(index=grid).reset_index()
model_data_full

Then we apply the model function with our determined best parameters and save the results in an additonal column in the data frame.

In [ ]:
model_data_6step["stress"] = sellars_tegart_model_6step(
    model_data_6step.strain, model_data_6step.rate, model_data_6step.temp, fit_6step.x
)
model_data_6step

In [ ]:
model_data_full["stress"] = sellars_tegart_model_full(
    model_data_full.strain, model_data_full.rate, model_data_full.temp, fit_full.x
)
model_data_full

To plot both in comparison, we first have to merge the data frames. We distinguish model predictions and experimental data by an additonal column containg a marker label.

In [ ]:
combined_data = pd.concat(
    [data, model_data_6step, model_data_full],  # list of the frames to combine
    keys=["exp", "6step", "full"],  # list of marker labels in same order as above
    names=["type", "index"],  # names of the marker column and the index column
)
combined_data.reset_index(
    level=0, inplace=True
)  # make the type column a normal column (was an index column)
combined_data.fillna(
    value={"file": ""}, inplace=True
)  # fill missing file in model results
combined_data

Now we plot the data as before, but distinguish the origin by lime style (solid and dashed).

In [ ]:
px.line(
    combined_data,
    "strain",
    "stress",
    color="temp",
    facet_col="rate",
    line_dash="type",
    line_group="file",
)